#Генерация текста

В этом уроке мы познакомимся с генерацией текста на практике, а именно -- реализуем языковую модель (то есть обучим модель генерировать текст заданном стиле). В качестве модели возьмём Char-RNN, то есть работать будем на уровне отдельных символов (букв).

В качестве обучающей выборки (корпуса) возьмём текст произведения Шекспира.
В результате наша обученная языковая модель должна генерировать текст в таком же стиле.

### Используем TensorFlow 2.0

На момент подготовки этих материалов в Google Colab по умолчанию используется версия TensorFlow 1.X

Переключаемся на версию 2.0 (работает только в Colab)

In [ ]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


### Загрузка библиотек
TensorFlow должен иметь как минимум версию 2.0

In [ ]:
import codecs
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.0.0


### Загрузка датасета

Скачиваем файл с текстом (`shakespeare.txt`) и загружаем его содержимое в переменную `text`.

Посмотрим, как выглядит текст, распечатав его фрагмент. Видно, что тексту свойственна некоторая стилистика пьесы.

In [ ]:
data_fpath = tf.keras.utils.get_file(
    'shakespeare.txt', 
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

text = codecs.open(data_fpath, 'r', encoding='utf8').read()
print(text[:250])

1122304/1115394 [==============================] - 0s 0us/step
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



### Конвертация символов в индексы

Как и раньше (как мы делали в случае классификации текстов), нам будет удобно работать с некоторым словарём и индексами слов в данном словаре. Только сейчас вместо слов мы будем испоьзовать символы (буквы итд).

Чтобы получить словарь `vocab` достаточно применить оператор `set` к нашему тексту (то есть конвертировать последовательность всех символов в множество = удалить все повторы). `VOCAB_SIZE` -- количество элементов в словаре.

Затем, как и раньше, создаём отображения символа в индекс и наоборот (`char2idx`, `idx2char`)

Теперь конвертируем наш текст в последовательность индексов с помощью `char2idx`

In [ ]:
vocab = sorted(set(text))
VOCAB_SIZE = len(vocab)

print('Vocab: {}'.format(vocab))
print('{} unique characters'.format(VOCAB_SIZE))

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

print('Example of the encoded text: {}'.format(text_as_int[:13]))

Vocab: ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65 unique characters
Example of the encoded text: [18 47 56 57 58  1 15 47 58 47 64 43 52]


### Подготовка датасета

Во время инференса наш генератор  будет работать следующим образом: сначала мы подаём входной символ или входную последовательность (зерно), получаем первый выходной символ, а затем подаём его как входной символ и так далее (генерируем по одному символу за раз).

А во время обучения будем обучать модель работать сразу с целой последовательностью. Например, обученная модель должна по входу `[F, i, r, s, ...]` выдавать `[i, r, s, t, ...]`. То есть ту же последовательность, но сдвинутую на 1 элемент. В данном случае это будет эквивалентно Many-to-Many Sync (только во время обучения).

Таким образом, зафиксируем длину рабочей цепочки `SEQ_LEN` (на которой будем обучаться), разрежем весь текст на цепочки длины `SEQ_LEN+1` (остаток отбросим), и из каждой такой цепочки длины `SEQ_LEN+1` получим пару цепочек длины `SEQ_LEN`, сдвинутых на 1 элемент: входная цепочка (без последнего элемента) и целевая (истинная) цепочка (начиная со второго элемента).

Кроме того, зафиксируем размер батча для обучения и отбросим цепочки в конце, которые не могут наполнить полный батч (чтоб кол-во обучающих цепочек было кратно размеру батча).


In [ ]:
SEQ_LEN = 1000
BATCH_SIZE = 64

input_seqs = []
target_seqs = []

num_seqs = len(text_as_int) // (SEQ_LEN+1)
for i in range(num_seqs):
    seq = text_as_int[i:i+SEQ_LEN+1]
    input_seqs.append(np.array(seq[:-1]))
    target_seqs.append(np.array(seq[1:]))

input_seqs = np.array(input_seqs)
target_seqs = np.array(target_seqs)

input_seqs = input_seqs[:(len(input_seqs)//BATCH_SIZE)*BATCH_SIZE]
target_seqs = target_seqs[:(len(input_seqs)//BATCH_SIZE)*BATCH_SIZE]

print('Input: {} ...'.format([idx2char[i] for i in input_seqs[0][:15]]))
print('Target: {} ...'.format([idx2char[i] for i in target_seqs[0][:15]]))

Input: ['F', 'i', 'r', 's', 't', ' ', 'C', 'i', 't', 'i', 'z', 'e', 'n', ':', '\n'] ...
Target: ['i', 'r', 's', 't', ' ', 'C', 'i', 't', 'i', 'z', 'e', 'n', ':', '\n', 'B'] ...


### Функция построения модели

Здесь мы создадим нашу модель с помощью `tf.keras.Sequential`.

Модель будет состоять из трёх слоёв:
* Embedding слой для отображения индексов букв в их вектрное представление
* Рекуррентный слой GRU
* Полносвязный слой, предсказывающий распределение по различным буквам (например, можно потом навесить Argmax, чтобы понять, какую именно букву предсказывает слой)

На выходе нам нужно получить последовательность такой же длины, что и входная,то есть чтоб GRU слой возвращал всю цепочку скрытых векторов, а не только последний. Для этого в параметрах GRU слоя зададим `return_sequences=True`.

Во время обучения будем просить модель предсказать входную цепочку, сдвинутую на 1 элемент. A во время инференса -- будем постепенно подавать символ за символом (то есть при каждом инференсе модель будет делать отображение "один в один"). Теоретически можно было бы и во время обучения делать так же -- постепенно генерировать выход (символ за символом), подавая результат предыдущей итерации (предыдущий символ) на вход в текущей итерации. Но сеть будет обучаться лучше, если мы будем "заставлять" её использовать "правильные" входы, а не то, что она сама нагенерировала. 

Чтобы во время инференса при различных запусках модели она помнила своё предыдущее состояние (иначе не получится использовать рекуррентность, ведь мы будем подавать последовательно последовательности из одного элемента), мы укажем флаг `stateful=True`

Кроме того, если модель имеет флаг `stateful=True`, ей нужно заранее знать размер батча (зададим через `batch_input_shape`). А так как у нас размер батча будет разный (для обучения >1, для инференса =1), нам понадобится создать модель два раза. Чтобы не дублировать код создания модели, обернём её создание в функцию `build_model`, которая принимает размер батча в качестве аргумента.

In [ ]:
def build_model(batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(VOCAB_SIZE, 256, batch_input_shape=(batch_size, None)),
        tf.keras.layers.GRU(256, return_sequences=True, stateful=True),
        tf.keras.layers.Dense(VOCAB_SIZE),
    ])
    model.build()
    return model

model = build_model(BATCH_SIZE)

### Обучение модели

На выходе модели у нас полносвязный слой без функции активации, то есть это просто логиты. По ним нам надо по сути сделать классификацию (номер класса = номер предсказанного символа). Для логитов и целевых (истинных) значений, представленных индексами, надо использовать лосс `SparseCategoricalCrossentropy(from_logits=True)`

Далее обучаем модель стандартным способом на наборах `(input_seqs, target_seqs)`

In [ ]:
EPOCHS = 100

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

history = model.fit(input_seqs, 
    target_seqs,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE)

Train on 1088 samples
Epoch 1/100
1088/1088 [==============================] - 8s 7ms/sample - loss: 3.7032
Epoch 2/100
1088/1088 [==============================] - 2s 2ms/sample - loss: 3.0807
Epoch 3/100
1088/1088 [==============================] - 2s 2ms/sample - loss: 2.8581
Epoch 4/100
1088/1088 [==============================] - 2s 2ms/sample - loss: 2.5706
Epoch 5/100
1088/1088 [==============================] - 2s 2ms/sample - loss: 2.3342
Epoch 6/100
1088/1088 [==============================] - 2s 2ms/sample - loss: 2.1832
Epoch 7/100
1088/1088 [==============================] - 2s 2ms/sample - loss: 2.0554
Epoch 8/100
1088/1088 [==============================] - 2s 2ms/sample - loss: 1.9277
Epoch 9/100
1088/1088 [==============================] - 2s 2ms/sample - loss: 1.7918
Epoch 10/100
1088/1088 [==============================] - 2s 2ms/sample - loss: 1.6416
Epoch 11/100
1088/1088 [==============================] - 2s 2ms/sample - loss: 1.4714
Epoch 12/100
1088/1088 [======

### Создание модели для инференса

После обучения модели нам надо создать такую же модель, но с размером батча =1, которую позже будем использовать для инференса (`model_inf`). Воспользуемся для этого функцией `build_model`, а затем скопируем обученные веса из `model` в новую модель `model_inf`.

In [ ]:
model_inf = build_model(1)

for i in range(len(model_inf.layers)):
    for j in range(len(model_inf.layers[i].weights)):
        model_inf.layers[i].weights[j].assign(model.layers[i].weights[j])

### Функция генерации текста

Создадим функцию, генерирующую текст по данной модели (`model`), входной цепочке (зерну `seed`) и желаемому количеству сгенерированных символов (`out_len`).

Внутри функции сначчала вызовем `model.reset_states()` для обнуления предыдущей истории состояния.

Затем конвертируем входной текст `seed` (символы в индексы) и прогоним его через нашу модель. Нас интересует результат, соответствующих только последнему выходу (символу). 

Как получить сам символ по предсказанию модели? Один из простейших способов -- просто взять argmax от предсказанных логитов (взять символ с наибольшей вероятностью). Однако, в таком случае выход часто будет слишком предсказуемым и иногда цепочка будет застрявать в цикличности (повторяться).

Для того, чтобы сделать предсказание менее предсказуемым, воспользуемся реальным распределенем и будем "сэмплировать" из него (то есть выбирать символ случайным образом в соответствии с его вероятностью). Сделать это можно с помощью функции `tf.random.categorical`. На вход в `categorical` мы подаём всю матрицу `pred` (у которой первое измерение это длина цепочки, а второе - распределение по классам), а на выходе получаем список сэмплов, по одному для каждого элемента последовательности). А так как нас интересует лишь последний символ, необходимо выбрать только его с помощью индекса `[-1]`.

Кроме того, можно ввести параметр (так называемая `температура`), с помощью которого можно управлять выходным распределением и таким образом влиять на непредсказуемость результата. Чем выше температура, тем более случайным будет предсказанный символ. Например, если мы разделим логиты на большое число (температуру), то если бы мы применили софтмакс (для получения вероятностей), то распределение стремилось бы к равномерному (у разных классов уравниваются шансы). 

Всё, что мы рассмотрели выше, соответствует первой итерации цикла `for`. Далее процесс повторяется, но теперь на входе каждый раз цепочка из одного символа (сгенерированного на предыдущей итерации). И делаем так, пока не соберем выходную цепочку длины `out_len`.

In [ ]:
def generate_text(model, seed, out_len):

    text_generated = []

    # Обнуляем состояние модели
    model.reset_states()
    
    # Конвертируем входную цепочку в индексы
    inp = np.array([char2idx[s] for s in seed])

    for i in range(out_len):

        # Получаем предсказания для входной цепочки inp
        # pred - матрица размерности (длина цепочки, распределение по классам)
        # На первой итерации цикла длина цепочки равна длине seed, а затем длина равна 1
        pred = model(inp[None, ...])[0]

        # Для получения символа сэмплируем из распределения
        # БОльшая температура соответствует более случайному предсказанию символа
        temperature = 1.0
        pred = pred / temperature
        pred_c = tf.random.categorical(pred, num_samples=1)[-1][0].numpy()
        
        text_generated.append(idx2char[pred_c])
        
        # Новый вход -- только что сгенерированный символ
        inp = np.array([pred_c])

    return (seed + ''.join(text_generated))

### Запуск генератора текста

Запускаем генерацию текста, передавая на вход желаемое начало цепочки `seed`.

In [ ]:
print(generate_text(model_inf, seed=u"MONTAGUE:", out_len=500))

MONTAGUE: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.

Second Citizen:
Would you proceed especially against Caius Marcius?

All:
Against him first: he's a very dog to the commonalty.

Second Citizen:
Consider you what services he has done for his country?

First Citizen:
Ve


**[Задание 1]** Поэксперементируйте с параметром `температура`. Найдите примеры значения параметра, при которых модель ведёт себя предсказуемо (детерминировано) и не предсказуемо (случайно).

**[Задание 2]** Попробуйте улучшить нейросеть за счёт увеличения её глубины и добавления регуляризации (dropout).

**[Задание 3]** Обучите модель Char-RNN на другом корпусе: возьмите датасет IMDB (из предыдущего модуля), объедените все отзывы в один текст и обучитесь на нём.

**[Задание 4]** Обучите модель синтеза текста на отдельных словах а не на символах. Используйте для этого датасет IMDB с ограниченным словарём `(num_words=...)`.

